# Option 2. Finding distance by focal length

In [2]:
import utils
import torch
import math
import numpy as np 

from PIL import Image
import cv2
import os
from pathlib import *
import shutil
import pandas as PD
import pillow_heif
import matplotlib.pyplot as plt

%matplotlib inline
PD.options.display.expand_frame_repr = False

In [4]:
display = utils.notebook_init() 
print('Cuda available?', torch.cuda.is_available())
print('At which number CUDA:', torch.cuda.current_device())
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # without this there will be an error

YOLOv5  2023-2-19 Python-3.9.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)


Setup complete  (12 CPUs, 32.0 GB RAM, 764.4/919.6 GB disk)
Cuda available? True
At which number CUDA: 0


In [5]:
# The function spins the picture depending on what it finds in the file characteristics. 
def exif_transpose(img):
    if not img:
        return img
    wh = img.size
    if wh[0] > wh[1]:
        return img
    exif_orientation_tag = 274

    # Check for EXIF data (only present on some files)
    if hasattr(img, "_getexif") and isinstance(img._getexif(), dict) and exif_orientation_tag in img._getexif():
        exif_data = img._getexif()
        orientation = exif_data[exif_orientation_tag]

        # Handle EXIF Orientation
        if orientation == 1:
            # Normal image - nothing to do!
            pass
        elif orientation == 2:
            # Mirrored left to right
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 3:
            # Rotated 180 degrees
            img = img.rotate(180)
        elif orientation == 4:
            # Mirrored top to bottom
            img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 5:
            # Mirrored along top-left diagonal
            img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 6:
            # Rotated 90 degrees
            #print('turned')
            img = img.rotate(-90, expand=True)
        elif orientation == 7:
            # Mirrored along top-right diagonal
            img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 8:
            # Rotated 270 degrees
            img = img.rotate(90, expand=True)

    return img

In [2]:
#model = torch.hub.load('ultralytics/yolov5', 'custom',  path='runs/train/bw/exp4/weights/last.pt') # works, but loads the BEST every time
model = torch.hub.load('ultralytics/yolov5', 'custom',  path='my_model/exp4/weights/best.pt') # works, but loads every time
model2 = torch.hub.load('ultralytics/yolov5', 'custom',  path='my_model/points/weights/best.pt') # works, but loads every time

Using cache found in C:\Users\Felix/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirements "gitpython" "tqdm>=4.64.0" "setuptools>=65.5.1" "wheel>=0.38.0" not found, attempting AutoUpdate...
requirements:  Command 'pip install "gitpython" "tqdm>=4.64.0" "setuptools>=65.5.1" "wheel>=0.38.0"  ' returned non-zero exit status 1.
YOLOv5  2023-2-19 Python-3.9.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)



Exception: [Errno 2] No such file or directory: 'my_model\\exp4\\weights\\best.pt'. Cache may be out of date, try `force_reload=True` or see https://github.com/ultralytics/yolov5/issues/36 for help.

In [5]:
def get_grayscale2(image):  # reduces the image by a factor of 3
    #https://habr.com/ru/post/163663/
    #convert image -colorspace gray image
    img = Image.open(image).convert('L')
    img.save(image)

In [6]:
from exif import Image as exif

def  get_focus_from_exif(f):
    with open(f, "rb") as palm_1_file:
        palm_1_image = exif(palm_1_file)
    if palm_1_image.has_exif:
        focal = palm_1_image.get('focal_length_in_35mm_film', 'Unknown')
        digital_zoom = palm_1_image.get('digital_zoom_ratio', 'Unknown')  #поиграем с зумом....а нет. зум начинает менять фокусное расстояние и он не нужен, но пусть будет пока.
    else:
        focal = 0
        digital_zoom = 1
    
    return focal, digital_zoom

In [7]:
# convert heic_to_jpg. inputs the source file and the path to the new file, returns the path to the new file
def conv_heic_to_jpg(file, new_name):

    heif_file = pillow_heif.read_heif(file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
    )

    ex = heif_file.info['exif']
    image.save(new_name, format("jpeg"), exif=ex)
    image.close()
    return new_name

In [8]:
#recognizing a file and retrieving data from it using a central number
def res(file, show_pandas = 0):
    if '.heic' in file:
        temp_file = 'tmp.jpg'  #нужен временный файл, чтобы не портить датасет
        file = conv_heic_to_jpg(work_file, temp_file)
    
    im = Image.open(file)
    (width, height) = im.size
    
    results = model(file)
    pd = results.pandas().xyxy[0]
 
    pd = pd.loc[pd['name'] == 'znak'] # sign table
    
    # We don't need all the rooms. We only need the one closest to the center

    # PROCESSING SIGNS
    if show_pandas == 1:
        print('Знаки\n',pd)
    
    pd = pd.assign(to_centre_x = abs(width/2 - (pd.xmin + (pd.xmax-pd.xmin)/2)))     #consider the distance to the center
    pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)                         #we calculate the coordinates of the center 
    
    pd = pd.assign(width = pd.xmax - pd.xmin)                                        #width
    pd = pd.assign(height = pd.ymax - pd.ymin)                                       #height
    pd = pd.assign(s2 = pd.width * pd.height)                                        #area
    
    pd = pd.sort_values(['to_centre_x'] )                                            #we sort by proximity to the center. We need the closest number
    
    
    d = dict()   # this dictionary will contain all the results
    for index, row in pd.iterrows():
        d ['width']  = row['width']
        d ['height']  = row['height']
        d ['s2']  = row['s2']
        d ['xmin']  = row['xmin']
        d ['ymin']  = row['ymin']
        d ['xmax']  = row['xmax']
        d ['ymax']  = row['ymax']
        break
    # Now in d are all the characteristics of the sign. 
    
    d['im_width'] = width
    d['im_height'] = height
    
    return d    

In [84]:
# procedure cuts out the sign, just to see what is found there. I needed it for debugging.
# input a file, and a dictionary with data about the sign
# WARNING: it will spoil the file it finds.

def crop_znak(temp_file,znak):
    if len(znak) <= 3:
        #print('не смог кроопнуть', temp_file)
        return
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    сentre_x = xmin + (xmax-xmin)/2
    сentre_y = ymin + (ymax-ymin)/2
    w_max = max(xmax-xmin, ymax-ymin)/2
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((сentre_x-w_max, сentre_y-w_max,сentre_x+w_max, сentre_y+w_max))
    img = img.resize((640,640))
    print('сохраняем в ',temp_file)
    img.save(temp_file)
    #считаем коэффициент кроп
    k = znak['width']/640
    return k

In [10]:
# Cut out the area around the sign to feed it back into the network for more accurate recognition. 
def mini_pic(temp_file,znak):
    if len(znak) == 3:
        print('не смог кроопнуть', temp_file)
        return
    if znak['width'] > 400: # only if the sign is very small
        return
    
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    centre_x = xmin + (xmax - xmin)/2
    centre_y = ymin + (ymax - ymin)/2
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
    #print('минипик',temp_file,centre_x,centre_y, znak['width'])
    
# try to cut out the area around the sign to feed it back to the network for more accurate recognition.  
# This is if the sign in the photo is small and it did not find anything at all. do a cropped center 960x960
def zoom960(temp_file):
    
    img = Image.open(temp_file)
    w,h = img.size
    centre_x = w//2
    centre_y = h//2
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
 

In [146]:
# Here we correct the sign: depending on the ratio between the found sides we find the angle and use it to calculate what 520 has become
    
from math import atan, sin, cos, tan,degrees

def correct_znak2(znak):
    
    w_znak = 529 #ширина знака в мм
    h_znak = 112
    if len(znak) == 3: #похоже, что не рааспознался
        return w_znak
    #сначала ищем угол наклона знака по соотношению сторон
    HW  = 112/520 #стандартное соотношение между сторонами
    hw = znak['height']/znak['width']#найденное соотнощение между сторонами
    tan_alfa = -(HW - hw) / (1 - HW * hw)
    alfa_rad = abs(math.atan(tan_alfa)) #нашли угол наклона знака в радианах
    print('приблизительно повернуто',degrees(alfa_rad))
    
    new_AB = w_znak * cos(alfa_rad) + h_znak * sin(alfa_rad)
    #new_w = (CD-AB/tan(alfa_rad)) / (sin(alfa_rad) - cos(alfa_rad)*cos(alfa_rad)/sin(alfa_rad))
    
    print('корректировка длины знака','520', '---->',new_AB)
    
    return new_AB


#старая версия, она корреткирует не 520, а найденное значение ширины уменьшает.
# def correct_znak(znak):
#     if len(znak) == 3: #похоже, что не рааспознался
#         return znak
#     #сначала ищем угол наклона знака по соотношению сторон
#     HW  = 112/520 #стандартное соотношение между сторонами
#     hw = znak['height']/znak['width']#найденное соотнощение между сторонами
#     tan_alfa = -(HW - hw) / (1 - HW * hw)
#     alfa_rad = math.atan(tan_alfa) #нашли угол наклона знака в радианах
#     print('приблизительно повернуто',degrees(alfa_rad))
    
#     CD = znak['height']
#     AB = znak['width']
#     new_w = (CD-AB/tan(alfa_rad)) / (sin(alfa_rad) - cos(alfa_rad)*cos(alfa_rad)/sin(alfa_rad))
    
#     print('корректировка длины знака',znak['width'], '---->',new_w)
#     znak['width'] = new_w
#     return znak
    

In [46]:
# THIS FUNCTION SEARCHES THE DISTANCE BETWEEN TWO fixing points on a number and returns it in pixels

def res2(file, show_pandas = 0):
#     if '.heic' in file:
#         temp_file = 'tmp.jpg'  #нужен временный файл, чтобы не портить датасет
#         file = conv_heic_to_jpg(work_file,temp_file)
    
    im = Image.open(file)
    (width, height) = im.size
    
    results = model2(file)
    pd = results.pandas().xyxy[0]
 
    pd = pd.loc[pd['name'] == 'znak']                   # sign table
    
    #все номера нам не надо. Нам надо только тот, что ближе к центру

    #ОБРАБАТЫВАЕМ точки на знаке
    if show_pandas == 1:
        print('Знаки\n',pd)
    
    pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)                         #считаем координаты центра 
    pd = pd.assign(centre_y = pd.ymin + (pd.ymax-pd.ymin)/2)                         #считаем координаты центра 
    
    pd = pd.assign(width = pd.xmax - pd.xmin)                                        #ширина
    pd = pd.assign(height = pd.ymax - pd.ymin)                                       #высота
    
    pd = pd.loc[pd['centre_y'] >280]
    pd = pd.loc[pd['centre_y'] <380]
    
    pd = pd.sort_values(['centre_x'] )                                            #сортируем по близости к центру. Нам надо самый близкий номер
    
    if show_pandas == 1:
        print('отсортировано\n',pd)
    d = []   #в этом будут две точки, нам надо найти расстояние между ними
    for index, row in pd.iterrows():
        d.append([row['centre_x'],row['centre_y']])
    if len(d) == 2:
        r = ((d[0][0] - d[1][0])**2 + (d[0][1] - d[1][1])**2) **0.5
    else:
        r = 0
         
    if show_pandas == 1:
        print('Расстояние', r)
    
    return r    

In [62]:
res2('Y:\\img_1931.jpg',1)

Знаки
          xmin        ymin        xmax        ymax  confidence  class  name
0  614.981262  300.257507  627.810974  313.202515    0.737279      0  znak
1   14.071442  325.275391   27.708603  339.688538    0.722411      0  znak
отсортировано
          xmin        ymin        xmax        ymax  confidence  ...  name    centre_x    centre_y      width     height
1   14.071442  325.275391   27.708603  339.688538    0.722411  ...  znak   20.890022  332.481964  13.637161  14.413147
0  614.981262  300.257507  627.810974  313.202515    0.737279  ...  znak  621.396118  306.730011  12.829712  12.945007

[2 rows x 11 columns]
Расстояние 601.0580124133401


601.0580124133401

In [154]:
def file_recognition(f):
    w_znak = 520 #ширина знака в мм
    #w = 4032    #разрешение матрицы/фото в пикселях
    
    name = f[f.rfind('/')+1:]
    work_file = f
    temp_file = 'temp.jpg'
    #temp_file = 'Y:/'+name
    if '.heic' in f:
        temp_file = 'Y:/'+name.replace('.heic','.jpg')
        print('Новое имя',temp_file)
        work_file = conv_heic_to_jpg(work_file,temp_file)
        #print('Сконвертирован в',work_file)
    focus, digital_zoom = get_focus_from_exif(work_file)
    print('focus, digital_zoom',focus, digital_zoom)
     
    img = Image.open(work_file)
    img = exif_transpose(img)
    w,h = img.size
    img.save(temp_file)
    
    img.close()

    #get_grayscale2(temp_file)  #пытался работать с серым. не очень.
    
    znak1 = res(temp_file)
    znak = znak1
    
#     #Заходим еще 1 раз, если ничего не нашли. просто на удачу
#     if len(znak) <= 3:
#         zoom960(temp_file)
#         znak1 = res(temp_file)
#         znak = znak1
        
    
    
    #ЭКСПЕРИМЕНТ
#     if len(znak1) <= 3:
#         print('ФАЙЛ НЕ РАСПОЗНАН', f)
#         not_recognize.append(f) 
#         return 0.0
    
#     #пробуем вырезать область,чтобы знак был бллиже. количество пикселей от этого не меняется
#     print('ширина до мини пика', znak1['width'])

#     mini_pic(temp_file,znak1)
#     #вырезали и еще раз на распознавание
#     znak = res(temp_file)
#     if len(znak) != 3:
#         print('ширина после мини пика', znak['width'], 'уточнение', znak1['width']-znak['width'])
#     else: #почему то после зума, знак не распознался. Возвращаем то, что было, это всяко лучше чем ноль
#         znak = znak1
        
    #....конец эксперимента.
    
    
    
   
    
    w_matrix = 35# 34.974 #ширина матрицы в мм. Вроде бы именнь это классическая ширина, но ее в разговоре просто округляют до 35 мм.
    #print('ширина кадра',w)
    pixel_in_mm = w/w_matrix
    d1 = 0
    if 'width' in znak:
        k = crop_znak(temp_file,znak)  #коэффикиент кропа
        r = res2(temp_file) #получили расстояние между двумя точками на номере 
        print('r,k',r* k,znak['width'])
        if r!=0:
            r = r * k #привели к нормальной ширине
            #print(r)
            d1 = (1 + 487 /(r/pixel_in_mm))* focus/1000
            print('расстояние по точкам', d1)
            return d1 #если есть расстояние между точками - за ним преимущество
    
    
    if len(znak) <= 3:
        print('ФАЙЛ НЕ РАСПОЗНАН', f)
        not_recognize.append(f)
        d = 0.0
    #формируем строку с параметрами знака для записи в csv
    else:
         #уточнение полученнного расстояние
        w_znak = correct_znak2(znak)
        d = (1 + w_znak /(znak['width']/pixel_in_mm))* focus/1000  #ну вот тут я заморочился. не мог решить до чего считать расстояние - до оптического центра или до матрицы. Стал считать до матрицы. Объектив может гулять от матрицы на много см.
    
    return d

In [167]:
train_csv = '../data_set/train.csv'  #data set if you run it on a training dataset, it will be considered an error.
keys = ['width', 'height', 's2', 'xmin', 'ymin', 'xmax', 'ymax','im_width', 'im_height']
pic_data = '../data_set/train'  # here is the dataset
pic_data_test  = pic_data

# TO RUN ON YOUR DATASET, YOU NEED TO CHANGE THIS PATH
pic_data_test = '../data_set/test'  # Here is the test dataset


test = '../data_set/sample_solution.csv'  # the result will be recorded here

# first read all the distances given to us from the training dataset
dist = dict()  # here we will store all distances extracted from the file. In the form of a dictionary
with open(train_csv, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.replace('\n','')
        key, d = line.split(';')
        dist[key] = d
        
    


new_f = 'image_name;distance'
not_recognize = []  # there will be some unrecognized
n = 0
abs_mistake, otn_mistake, vsego = 0, 0, 0  # To calculate the error
for f in os.listdir(pic_data_test):
    n +=1
    if n<= 380:
        continue
    if n>385:
         break
    
    
    work_file = os.path.join(pic_data_test, f)
    print(n, 'working with the file: ', work_file)

    itog = file_recognition(work_file)
    
    # we get the true value, count the error
    if f in dist:
        y = float(dist[f])
        mistake = (y - itog) / y # I apologize for not using your mistake, but it makes more sense to me in percentages, and since I don't train on a mistake, I did so.
        print('Error', mistake, 'estimation', y, 'предск', itog)
        otn_mistake += mistake  #you need it to see if it goes in the plus or minus
        abs_mistake += abs(mistake)
        vsego +=1
    
    
    st = f+';' + str(itog)
    print('Recorded result: ',st)
    print('\n\n')    
    new_f = new_f +'\n'+st
    
#print(new_f)
with open(test, 'w', encoding = 'utf-8') as file:
    file.write(new_f)
print('Nr of files not detected:', len(not_recognize))
print('unrecognized:', *not_recognize)
print('saved in:', test)
if vsego != 0:
    print('Аbsolute error', round(abs_mistake/vsego*100, 5))
    print('Relative error', round(otn_mistake/vsego*100, 5))
else:
    print('No error has been calculated. No true data detected. Is the dataset a test one?')

381 работаем  с файлом:  ..\data_set\test\img_2674.heic
Новое имя Y:\img_2674.jpg
focus, digital_zoom 14 1.0327868852459017
ФАЙЛ НЕ РАСПОЗНАН ..\data_set\test\img_2674.heic
Записан результат:  img_2674.heic;0.0



382 работаем  с файлом:  ..\data_set\test\img_2675.jpg
focus, digital_zoom 14 1.0223123732251522
сохраняем в  temp.jpg
r,k 148.69137140255333 158.6239013671875
расстояние по точкам 5.296307860848156
Записан результат:  img_2675.jpg;5.296307860848156



383 работаем  с файлом:  ..\data_set\test\img_2676.jpg
focus, digital_zoom 14 1.0223123732251522
сохраняем в  temp.jpg
r,k 189.8632736006027 206.633056640625
расстояние по точкам 4.150837973478968
Записан результат:  img_2676.jpg;4.150837973478968



384 работаем  с файлом:  ..\data_set\test\img_2677.heic
Новое имя Y:\img_2677.jpg
focus, digital_zoom 14 1.0327868852459017
сохраняем в  Y:\img_2677.jpg
r,k 139.3193845650131 157.6392822265625
расстояние по точкам 5.651647642876854
Записан результат:  img_2677.heic;5.65164764287685

In [56]:
res('Y:\\255_1.jpg',1)

Знаки
           xmin         ymin         xmax         ymax  confidence  class  name
0  1760.632202  1313.192139  2381.237305  1438.997437    0.952805      0  znak


{'width': 620.6051025390625,
 'height': 125.8052978515625,
 's2': 78075.40977312624,
 'xmin': 1760.6322021484375,
 'ymin': 1313.192138671875,
 'xmax': 2381.2373046875,
 'ymax': 1438.9974365234375,
 'im_width': 3968,
 'im_height': 2976}

In [166]:
file_recognition('..\\data_set\\test\\img_2674.heic')

Новое имя Y:\img_2674.jpg
focus, digital_zoom 14 1.0327868852459017
ФАЙЛ НЕ РАСПОЗНАН ..\data_set\test\img_2674.heic


0.0

In [161]:
print(file_recognition('Y:\\255_1.jpg'))

print('\n')
print(file_recognition('Y:\\255_2.jpg'))


print('\n')

print(file_recognition('Y:\\255_3.jpg'))


print('\n')

focus, digital_zoom 27 1.0
сохраняем в  temp.jpg
r,k 592.7320572254057 619.9266357421875
расстояние по точкам 2.5419996463221812
2.5419996463221812


focus, digital_zoom 31 1.0
сохраняем в  temp.jpg
r,k 718.0005061784991 743.150390625
расстояние по точкам 2.414798399046464
2.414798399046464


focus, digital_zoom 52 1.0
сохраняем в  temp.jpg
r,k 1166.430252539312 1251.6114501953125
расстояние по точкам 2.513371394382705
2.513371394382705




In [160]:
print(file_recognition('Y:\\421_1.jpg'))
print('\n')
print(file_recognition('Y:\\421_2.jpg'))
print('\n')

print(file_recognition('Y:\\421_3.jpg'))
print('\n')

FileNotFoundError: [Errno 2] No such file or directory: 'Y:\\421_1.jpg'

In [91]:
print(file_recognition('Y:\\490_1.jpg'))
print('\n')
print(file_recognition('Y:\\490_2.jpg'))
print('\n')

print(file_recognition('Y:\\490_3.jpg'))
print('\n')

4.923146672370446


4.690990338609638


4.974977451052313


